In [2]:
from typing import Callable
import numpy as np
from numpy import ndarray
from typing import List

In [3]:
def sigmoid(x: ndarray) -> ndarray:
    return 1 / (1 + np.exp(-x))

def square(x: ndarray) -> ndarray:
    return np.power(x, 2)

In [10]:
# производная
def deriv(func: Callable[[ndarray], ndarray], input_:ndarray,delta: float = 0.001) -> ndarray:
    return (func(input_+ delta) - func(input_ - delta)) / (2 * delta)

deriv(square, np.array([10, 20, 30]))

array([20., 40., 60.])

In [7]:
# вложенные функции
Array_Function = Callable[[ndarray], ndarray]
Chain = List[Array_Function]

def chain_length_2(chain: Chain, x: ndarray) -> ndarray:
    f1 = chain[0]
    f2 = chain[1]
    return f2(f1(x))

list_func = [square, square]
chain_length_2(list_func, np.array([2,4,8]))

array([  16,  256, 4096], dtype=int32)

In [25]:
def chain_deriv_3(chain: Chain,
                  input_range: ndarray) -> ndarray:

    f1 = chain[0]
    f2 = chain[1]
    f3 = chain[2]

    # f1(x)
    f1_of_x = f1(input_range)

    # f2(f1(x))
    f2_of_x = f2(f1_of_x)

    # df3du
    df3du = deriv(f3, f2_of_x)

    # df2du
    df2du = deriv(f2, f1_of_x)

    # df1dx
    df1dx = deriv(f1, input_range)
    return df1dx * df2du * df3du

list_func = [sigmoid, square, square]
chain_deriv_3(list_func, np.array([2,4,5]))

array([0.28697917, 0.06690677, 0.02606186])

In [33]:
def multiple_inputs_add_backward(x: ndarray,
                                 y: ndarray,
                                 sigma: Array_Function) -> float:
    a = x + y

    dsda = deriv(sigma, a)

    dadx, dady = 1, 1

    return dsda * dadx, dsda * dady

multiple_inputs_add_backward(np.array([4,5,6]), np.array([8,9,3]), sigmoid)

(array([6.14413792e-06, 8.31527458e-07, 1.23379370e-04]),
 array([6.14413792e-06, 8.31527458e-07, 1.23379370e-04]))

In [11]:
def matrix_function_backward_sum_1(X: ndarray,
                                W: ndarray,
                                sigma: Array_Function) -> ndarray:

    N = np.dot(X, W)
    S = sigma(N)
    L = np.sum(S)

    dLdS = np.ones_like(S)
    dSdN = deriv(sigma, N)
    dLdN = dLdS * dSdN
    dNdX = np.transpose(W, (1, 0))
    dLdX = np.dot(dLdN, dNdX)

    return dLdX

In [12]:
np.random.seed(190204)
X = np.random.randn(3, 3)
W = np.random.randn(3, 2)

matrix_function_backward_sum_1(X, W , sigmoid)

array([[ 0.2488887 , -0.37478057,  0.01121962],
       [ 0.12604152, -0.27807404, -0.13945837],
       [ 0.22992798, -0.36623443, -0.02252592]])